## Check how AROMA edge fraction feature is calculated

Link to their GitHub repo
https://github.com/maartenmennes/ICA-AROMA/blob/master/ICA_AROMA_functions.py

In [ ]:
import numpy as np
import os
import sys
import subprocess
from past.utils import old_div

# hardcoded paths
tempDir = '/LOCAL/jzerbe/faces_vs_houses'
#'/LOCAL/jzerbe/faces_vs_houses'
melDir = '/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/melodic/sub-01/sub-01_task-1back_space-MNI152NLin6Asym_melodic'
#'/LOCAL/jzerbe/faces_vs_houses/ds002938/derivatives/melodic/sub-01/sub-01_task-1back_space-MNI152NLin2009cAsym_melodic'

melIC = os.path.join(melDir, 'melodic_IC.nii.gz')
fslDir = os.path.join(os.environ["FSLDIR"], 'bin', '')

# Get the number of ICs
numICs = int(subprocess.getoutput('%sfslinfo %s | grep dim4 | head -n1 | awk \'{print $2}\'' % (fslDir, melIC) ))
# Loop over ICs
edgeFract = np.zeros(numICs)
csfFract = np.zeros(numICs)

In [ ]:
for i in range(0, numICs):
    # Define temporary IC-file
    tempIC = os.path.join(tempDir, 'temp_IC.nii.gz')
    # Extract IC from the merged melodic_IC_thr2MNI2mm file
    os.system(' '.join([os.path.join(fslDir, 'fslroi'),
              melIC,
              tempIC,
              str(i),
              '1']))
    # Change to absolute Z-values
    os.system(' '.join([os.path.join(fslDir, 'fslmaths'),
              tempIC,
              '-abs',
              tempIC]))
    # Get sum of Z-values within the total Z-map (calculate via the mean and number of non-zero voxels)
    totVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                tempIC,
                                                '-V | awk \'{print $1}\''])))
    totMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                           tempIC,
                                                           '-M'])))
    totSum = totMean * totVox
    
    # Get sum of Z-values of the voxels located within the CSF (calculate via the mean & number of non-zero voxels)
    csfVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                #tempIC,
                                                os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_csf.nii.gz'),
                                                #'-k mask_csf.nii.gz',
                                                '-V | awk \'{print $1}\''])))
    csfMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                     #tempIC,
                                                     os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_csf.nii.gz'),
                                                     #'-k mask_csf.nii.gz',
                                                     '-M'])))
    csfSum = csfMean * csfVox
    print("csfVox", csfVox)
    print("csfMean", csfMean)
    print("csfSum", csfSum)
    
    # Get sum of Z-values of the voxels located within the Edge (calculate via the mean & number of non-zero voxels)
    edgeVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                     #tempIC,
                                                     os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_edge.nii.gz'),
                                                     #'-k mask_edge.nii.gz',
                                                         '-V | awk \'{print $1}\''])))
                  
    edgeMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                    #tempIC,
                                                    os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_edge.nii.gz'),
                                                    #'-k mask_edge.nii.gz',
                                                    '-M'])))
    
    edgeSum = edgeMean * edgeVox
    print("edgeVox", edgeVox)
    print("edgeMean", edgeMean)
    print("edgeSum", edgeSum)
    sys.exit()
    
    # Get sum of Z-values of voxels located outside the brain (calculate via the mean & number of non-zero voxels)
    outVox = int(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                #tempIC,
                                                os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_out.nii.gz'),
                                                #'-k mask_out.nii.gz',
                                                '-V | awk \'{print $1}\''])))
    outMean = float(subprocess.getoutput(' '.join([os.path.join(fslDir, 'fslstats'),
                                                #tempIC,
                                               os.path.join('/LOCAL/jzerbe/faces_vs_houses', 'mask_out.nii.gz'),
                                               #'-k mask_out.nii.gz',
                                               '-M'])))
    outSum = outMean * outVox
    
    # Determine edge and CSF fraction
    if not (totSum == 0):
        edgeFract[i] = old_div((outSum + edgeSum), (totSum - csfSum))
        csfFract[i] = old_div(csfSum, totSum)
    else:
        edgeFract[i] = 0
        csfFract[i] = 0
    
    # Remove the temporary IC-file
    os.remove(tempIC)
    

In [ ]:
# auditory
print(edgeFract)

In [ ]:
# faces vs houses
print(edgeFract)